In [1]:
# for getting data
import os
import zipfile

# for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline

### Crashes

In [2]:
crashes = pd.read_csv('./data/traffic_crashes.csv', low_memory = False)
crashes.head()

,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,NaN,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,NaN,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,1.0,0.0,14,7,7,41.854120,-87.665902,POINT (-87.665902342962 41.854120262952)
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,NaN,08/18/2023 05:58:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,...,1.0,0.0,1.0,0.0,17,6,8,41.942976,-87.761883,POINT (-87.761883496974 41.942975745006)
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,NaN,11/26/2019 08:38:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,ONE-WAY,...,0.0,0.0,1.0,0.0,8,3,11,NaN,NaN,NaN
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,NaN,08/18/2023 10:45:00 AM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,OTHER,...,0.0,0.0,1.0,0.0,10,6,8,NaN,NaN,NaN


In [3]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901446 entries, 0 to 901445
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                901446 non-null  object 
 1   CRASH_DATE_EST_I               66531 non-null   object 
 2   CRASH_DATE                     901446 non-null  object 
 3   POSTED_SPEED_LIMIT             901446 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         901446 non-null  object 
 5   DEVICE_CONDITION               901446 non-null  object 
 6   WEATHER_CONDITION              901446 non-null  object 
 7   LIGHTING_CONDITION             901446 non-null  object 
 8   FIRST_CRASH_TYPE               901446 non-null  object 
 9   TRAFFICWAY_TYPE                901446 non-null  object 
 10  LANE_CNT                       199022 non-null  float64
 11  ALIGNMENT                      901446 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [4]:
crashes.columns = crashes.columns.str.lower()

In [5]:
# Convert all string values in object columns to lowercase
for col in crashes.select_dtypes(include='object').columns:
    crashes[col] = crashes[col].str.lower()

In [6]:
missing_percentage = round((crashes.isna().sum()/len(crashes)*100), 2)
missing_percentage

crash_record_id                   0.00
crash_date_est_i                 92.62
crash_date                        0.00
posted_speed_limit                0.00
traffic_control_device            0.00
device_condition                  0.00
weather_condition                 0.00
lighting_condition                0.00
first_crash_type                  0.00
trafficway_type                   0.00
lane_cnt                         77.92
alignment                         0.00
roadway_surface_cond              0.00
road_defect                       0.00
report_type                       3.11
crash_type                        0.00
intersection_related_i           77.03
not_right_of_way_i               95.45
hit_and_run_i                    68.64
damage                            0.00
date_police_notified              0.00
prim_contributory_cause           0.00
sec_contributory_cause            0.00
street_no                         0.00
street_direction                  0.00
street_name              

In [7]:
# selecting all features with 90% or more of its values are null
high_null_features = crashes.columns[(crashes.isna().sum() / len(crashes) * 100) >= 90]
high_null_features

Index(['crash_date_est_i', 'not_right_of_way_i', 'photos_taken_i',
       'statements_taken_i', 'dooring_i', 'work_zone_i', 'work_zone_type',
       'workers_present_i'],
      dtype='object')

In [8]:
# creating a list of features with 90% or more null values
high_null_features_list = list(high_null_features)
high_null_features_list

['crash_date_est_i',
 'not_right_of_way_i',
 'photos_taken_i',
 'statements_taken_i',
 'dooring_i',
 'work_zone_i',
 'work_zone_type',
 'workers_present_i']

In [9]:
crashes_cleaned = crashes.drop(columns=high_null_features_list)


In [10]:
medium_null_features = crashes.columns[
    ((crashes.isna().sum() / len(crashes) * 100) >= 60) &
    ((crashes.isna().sum() / len(crashes) * 100) < 90)
]

In [11]:
medium_null_features_list = list(medium_null_features)

In [12]:
for feature in medium_null_features_list:
    print(f"Value counts for column '{feature}':")
    print(crashes_cleaned[feature].value_counts())
    print("-"* 32)

Value counts for column 'lane_cnt':
2.0          91162
4.0          49589
1.0          32550
3.0           8678
0.0           8032
6.0           4502
5.0           1940
8.0           1908
7.0            184
10.0           162
99.0           108
9.0             66
11.0            30
12.0            29
20.0            15
22.0            13
15.0             7
16.0             7
14.0             5
30.0             5
40.0             4
60.0             3
21.0             3
25.0             2
100.0            2
902.0            1
24.0             1
80.0             1
218474.0         1
45.0             1
17.0             1
299679.0         1
19.0             1
400.0            1
13.0             1
1191625.0        1
35.0             1
433634.0         1
41.0             1
28.0             1
44.0             1
Name: lane_cnt, dtype: int64
--------------------------------
Value counts for column 'intersection_related_i':
y    197181
n      9873
Name: intersection_related_i, dtype: int64
------

hit_and_run_i is an aftermath, not a contributor so we can remove that

lane_cnt could be important. Will need to reduce cardinality though.

intersection_related_i... could be helpful. A little vague and subjective according to CDOT description. "A field observation by the police officer whether an intersection played a role in the crash. Does not represent whether or not the crash occurred within the intersection."

In [13]:
crashes_cleaned['crash_date'].value_counts()

12/29/2020 05:00:00 pm    30
11/10/2017 10:30:00 am    27
02/17/2022 03:30:00 pm    21
11/21/2024 10:30:00 am    20
11/21/2024 10:00:00 am    20
                          ..
12/23/2016 12:41:00 pm     1
10/03/2020 05:32:00 pm     1
08/02/2021 05:15:00 pm     1
01/08/2020 02:35:00 pm     1
09/13/2023 01:08:00 pm     1
Name: crash_date, Length: 592919, dtype: int64

Will remove this feature. This information is captured in crash_hour, crash_day_of_the_week, crash_month

In [14]:
crashes_cleaned['posted_speed_limit'].value_counts()

30    664045
35     59626
25     57789
20     37717
15     32112
10     21096
40      8612
0       7584
45      5951
5       4957
55       883
50       276
3        221
9         96
39        95
99        66
60        53
1         41
24        38
2         31
65        20
32        20
34        16
33        14
11        11
26        11
36         8
6          7
70         7
7          6
18         4
12         4
22         4
14         4
23         3
29         3
31         2
8          2
38         2
16         2
4          2
62         1
63         1
44         1
49         1
46         1
Name: posted_speed_limit, dtype: int64

This feature could be important, but will require cardinality reduction

In [15]:
crashes_cleaned['traffic_control_device'].value_counts()

no controls                 510287
traffic signal              249882
stop sign/flasher            89361
unknown                      38328
other                         6096
yield                         1365
lane use marking              1226
other reg. sign               1103
other warning sign             715
pedestrian crossing sign       636
railroad crossing gate         581
flashing control signal        373
school zone                    353
delineators                    352
police/flagman                 309
rr crossing sign               195
other railroad crossing        192
no passing                      58
bicycle crossing sign           34
Name: traffic_control_device, dtype: int64

In [16]:
crashes_cleaned['device_condition'].value_counts()

no controls                 516329
functioning properly        307784
unknown                      63428
other                         6836
functioning improperly        4113
not functioning               2562
worn reflective material       295
missing                         99
Name: device_condition, dtype: int64

The top two categories that make up the majority of this feature are no controls and functioning properly. And then the next two frequent are unknown and other. We can drop this. won't be useful for analysis

In [17]:
crashes_cleaned['weather_condition'].value_counts()

clear                       709235
rain                         77962
unknown                      51500
snow                         28844
cloudy/overcast              26333
other                         2789
freezing rain/drizzle         1787
fog/smoke/haze                1353
sleet/hail                    1026
blowing snow                   453
severe cross wind gate         156
blowing sand, soil, dirt         8
Name: weather_condition, dtype: int64

In [18]:
crashes_cleaned['lighting_condition'].value_counts()

daylight                  578548
darkness, lighted road    197098
unknown                    42569
darkness                   42455
dusk                       25737
dawn                       15039
Name: lighting_condition, dtype: int64

In [19]:
crashes_cleaned['first_crash_type'].value_counts()

parked motor vehicle            208646
rear end                        199321
sideswipe same direction        138501
turning                         129668
angle                            97996
fixed object                     41874
pedestrian                       21320
pedalcyclist                     14331
sideswipe opposite direction     12509
rear to front                     9252
other object                      8981
head on                           7639
rear to side                      5512
other noncollision                2745
rear to rear                      1907
animal                             655
overturned                         543
train                               46
Name: first_crash_type, dtype: int64

This seems like it could be useful. 

In [20]:
crashes_cleaned['trafficway_type'].value_counts()

not divided                        388246
divided - w/median (not raised)    142466
one-way                            114072
four way                            62561
parking lot                         61011
divided - w/median barrier          50946
other                               24388
alley                               14802
t-intersection                      12409
unknown                             10603
center turn lane                     6374
driveway                             2890
ramp                                 2834
unknown intersection type            2762
five point, or more                  1385
y-intersection                       1350
traffic route                        1166
not reported                          687
roundabout                            308
l-intersection                        186
Name: trafficway_type, dtype: int64

This is important will keep this

In [21]:
crashes_cleaned['alignment'].value_counts()

straight and level       880103
straight on grade         11022
curve, level               6352
straight on hillcrest      2267
curve on grade             1313
curve on hillcrest          389
Name: alignment, dtype: int64

While this feature would ideally be helpful in analysis, the data here is not conducive for analysis. Most of the entries are 'straight and level'. will remove.

In [22]:
crashes_cleaned['roadway_surface_cond'].value_counts()

dry                667224
wet                117323
unknown             80085
snow or slush       28524
ice                  5678
other                2290
sand, mud, dirt       322
Name: roadway_surface_cond, dtype: int64

This is somewhat redundant with weather condition. Will remove weather_condition  and keep roadway_surface_cond due to its lower cardinality

In [23]:
crashes_cleaned['road_defect'].value_counts()

no defects           718022
unknown              166233
rut, holes             6350
other                  4893
worn surface           3741
shoulder defect        1547
debris on roadway       660
Name: road_defect, dtype: int64

The main two categories here are "no defects" and unknown. This will not be helpful for analysis. Will remove

In [24]:
crashes_cleaned['crash_type'].value_counts()

no injury / drive away              658842
injury and / or tow due to crash    242604
Name: crash_type, dtype: int64

This describes the aftermath, not helpful for contributory factors. Will remove

In [25]:
crashes_cleaned['beat_of_occurrence'].value_counts()

1834.0    10913
114.0      9281
813.0      9093
815.0      8590
1831.0     8244
          ...  
1653.0      502
1655.0      313
1652.0      241
1650.0       69
6100.0        7
Name: beat_of_occurrence, Length: 276, dtype: int64

High cardinality, unlikely to be useful for analysis. to remove

In [26]:
crashes_cleaned['most_severe_injury'].value_counts()

no indication of injury     772801
nonincapacitating injury     71130
reported, not evident        39463
incapacitating injury        15074
fatal                          985
Name: most_severe_injury, dtype: int64

In [27]:
crashes_cleaned['injuries_total'].value_counts()

0.0     772815
1.0      95189
2.0      21269
3.0       6479
4.0       2302
5.0        825
6.0        325
7.0        133
8.0         53
9.0         27
10.0        16
11.0         9
15.0         8
12.0         6
21.0         4
13.0         3
17.0         1
14.0         1
19.0         1
16.0         1
Name: injuries_total, dtype: int64

In [28]:
crashes_cleaned['injuries_fatal'].value_counts()

0.0    898482
1.0       912
2.0        64
3.0         8
4.0         1
Name: injuries_fatal, dtype: int64

In [29]:
crashes_cleaned['injuries_incapacitating'].value_counts()

0.0     884243
1.0      13370
2.0       1395
3.0        312
4.0        107
5.0         29
6.0          7
7.0          2
10.0         1
8.0          1
Name: injuries_incapacitating, dtype: int64

The 'injuries_...' features are redundant. This information is captured in the 'most_severe_injury' feature. Will remove all 'injuries_...' features and keep most_severe_injury. 

In [30]:
crashes_cleaned['prim_contributory_cause'].value_counts()

unable to determine                                                                 352689
failing to yield right-of-way                                                        99589
following too closely                                                                86950
not applicable                                                                       47632
improper overtaking/passing                                                          44963
failing to reduce speed to avoid crash                                               37868
improper backing                                                                     34796
improper lane usage                                                                  32108
driving skills/knowledge/experience                                                  30632
improper turning/no signal                                                           30203
disregarding traffic signals                                                         17608

In [31]:
crashes_cleaned['sec_contributory_cause'].value_counts()

not applicable                                                                      371652
unable to determine                                                                 324878
failing to reduce speed to avoid crash                                               33161
failing to yield right-of-way                                                        28925
driving skills/knowledge/experience                                                  28101
following too closely                                                                23735
improper overtaking/passing                                                          14021
improper lane usage                                                                  12692
weather                                                                               9915
improper turning/no signal                                                            9382
improper backing                                                                      7194

This feature is redundant to prim_contributory_cause. A high majority of values are either not applicable or unable to determine so it will be dropped. 

Damage, 'date_police_notified' both deal with aftermath, Not contributory factors. To be removed

'street_no'... unhelpful. Will remove

'num_units' not helpful in contributory factor, to remove

remove latitude and longitude as these are captured in location feature. 

In [32]:
crash_features_to_remove = [
    'crash_date',
    'hit_and_run_i',
    'device_condition',
    'weather_condition',
    'road_defect',
    'crash_type',
    'damage',
    'date_police_notified',
    'sec_contributory_cause',
    'street_no',
    'report_type',
    'beat_of_occurrence',
    'num_units',
    'alignment',
    'injuries_total',
    'injuries_fatal',
 'injuries_incapacitating',
 'injuries_non_incapacitating',
 'injuries_reported_not_evident',
 'injuries_no_indication',
 'injuries_unknown',
    'latitude',
 'longitude',   
]

In [33]:
crashes_cleaned = crashes_cleaned.drop(columns=crash_features_to_remove)
crashes_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901446 entries, 0 to 901445
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   crash_record_id          901446 non-null  object 
 1   posted_speed_limit       901446 non-null  int64  
 2   traffic_control_device   901446 non-null  object 
 3   lighting_condition       901446 non-null  object 
 4   first_crash_type         901446 non-null  object 
 5   trafficway_type          901446 non-null  object 
 6   lane_cnt                 199022 non-null  float64
 7   roadway_surface_cond     901446 non-null  object 
 8   intersection_related_i   207054 non-null  object 
 9   prim_contributory_cause  901446 non-null  object 
 10  street_direction         901442 non-null  object 
 11  street_name              901445 non-null  object 
 12  most_severe_injury       899453 non-null  object 
 13  crash_hour               901446 non-null  int64  
 14  cras

### People

In [34]:
people = pd.read_csv('./data/people.csv', low_memory = False)

In [35]:
people.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [36]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979859 entries, 0 to 1979858
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PERSON_ID              object 
 1   PERSON_TYPE            object 
 2   CRASH_RECORD_ID        object 
 3   VEHICLE_ID             float64
 4   CRASH_DATE             object 
 5   SEAT_NO                float64
 6   CITY                   object 
 7   STATE                  object 
 8   ZIPCODE                object 
 9   SEX                    object 
 10  AGE                    float64
 11  DRIVERS_LICENSE_STATE  object 
 12  DRIVERS_LICENSE_CLASS  object 
 13  SAFETY_EQUIPMENT       object 
 14  AIRBAG_DEPLOYED        object 
 15  EJECTION               object 
 16  INJURY_CLASSIFICATION  object 
 17  HOSPITAL               object 
 18  EMS_AGENCY             object 
 19  EMS_RUN_NO             object 
 20  DRIVER_ACTION          object 
 21  DRIVER_VISION          object 
 22  PHYSICAL_CONDITION

In [37]:
people.columns = people.columns.str.lower()

In [38]:
# Convert all string values in object columns to lowercase
for col in people.select_dtypes(include='object').columns:
    people[col] = people[col].str.lower()

In [39]:
round((people.isna().sum()/ len(people)*100), 2)

person_id                 0.00
person_type               0.00
crash_record_id           0.00
vehicle_id                2.05
crash_date                0.00
seat_no                  79.78
city                     27.23
state                    26.13
zipcode                  33.05
sex                       1.68
age                      29.11
drivers_license_state    41.46
drivers_license_class    51.34
safety_equipment          0.28
airbag_deployed           1.98
ejection                  1.26
injury_classification     0.04
hospital                 83.81
ems_agency               90.01
ems_run_no               98.33
driver_action            20.40
driver_vision            20.43
physical_condition       20.34
pedpedal_action          98.04
pedpedal_visibility      98.04
pedpedal_location        98.04
bac_result               20.35
bac_result value         99.89
cell_phone_use           99.94
dtype: float64

In [ ]:
for feature in people.columns:
    print(f"Value counts for column '{feature}':")
    print(people[feature].value_counts())
    
    # Print the number of unique values in the feature
    num_unique = people[feature].nunique()
    print(f"Number of unique values: {num_unique}")
    
    print("\n")

In [ ]:
# Replace string 'nan' with np.nan in the entire DataFrame
people.replace('nan', np.nan, inplace=True)

In [ ]:
round((people.isna().sum()/ len(people)*100), 2)

* remove city, state, zip, drivers_license_state, drivers_license_class ejection, hospital, ems_agency, ems_run_no, ejection; not relevant
* remove potentially injury_classification
* bac_result, bac_result_value, cell_phone_use... too many unknown or missing values
* crash_date to datetime
* convert dtypes from int to str

In [ ]:
ppl_features_to_remove = ['city', 
                          'state', 
                          'zipcode', 
                          'drivers_license_state',
                          'ejection', 
                          'hospital',
                          'ems_agency',
                          'ems_run_no',
                          'bac_result',
                          'bac_result value', 
                          'cell_phone_use',
                          'drivers_license_class',
                          'seat_no',
                          'airbag_deployed'
                         ]


In [ ]:
ppl_cleaned = people.drop(columns=ppl_features_to_remove)
ppl_cleaned.info()

In [ ]:
ppl_missing_percentage = round((ppl_cleaned.isna().sum()/ len(ppl_cleaned)*100), 2)

In [ ]:
# Filter columns with more than 80% missing values
ppl_columns_to_drop = ppl_missing_percentage[ppl_missing_percentage > 80].index

# Drop these columns from the dataframe
ppl_cleaned = ppl_cleaned.drop(columns=ppl_columns_to_drop)

# Display the cleaned dataframe (optional)
ppl_cleaned.info()

In [ ]:
round((ppl_cleaned.isna().sum()/ len(ppl_cleaned)*100), 2)

In [ ]:
# Set pandas option to display all rows in value_counts()
pd.set_option('display.max_rows', 60)

ppl_cleaned.age.value_counts()

In [ ]:
# Replace 0 values with NaN in the specific column
ppl_cleaned['age'] = ppl_cleaned['age'].replace(0, np.nan)

# Remove rows where the column has negative values
ppl_cleaned = ppl_cleaned[ppl_cleaned['age'] >= 0]

In [ ]:
# Bucketing ages

# Define the bins for age groups relevant to drivers
age_bins = [0, 16, 26, 65, 100]  # Legal driving age (16), Young drivers (16-26), Adult drivers (27-65), Older drivers (65+)
age_labels = ['Under 16', '16-26', '27-65', '65+']  # Labels for the age groups

# Ensure that you're using .loc to assign the new column
ppl_cleaned.loc[:, 'age_group'] = pd.cut(ppl_cleaned['age'], bins=age_bins, labels=age_labels, right=False)

In [ ]:
# Print the first few rows to verify
ppl_cleaned['age_group'].value_counts()

In [ ]:
for feature in ppl_cleaned.columns:
    print(f"Value counts for column '{feature}':")
    print(ppl_cleaned[feature].value_counts())
    
    # Print the number of unique values in the feature
    num_unique = ppl_cleaned[feature].nunique()
    print(f"Number of unique values: {num_unique}")
    
    print("\n")

convert to date_time = 'crash_date'

In [ ]:
# List of columns to convert
change_to_str = ['age_group', 
                 'physical_condition',
                 'driver_vision',
                 'driver_action',
                 'injury_classification',
                 'safety_equipment',
                 'sex']

# Convert the selected columns to string
ppl_cleaned[change_to_str] = ppl_cleaned[change_to_str].astype(str)

# Verify the changes
ppl_cleaned.info()

In [ ]:
(ppl_cleaned.isna().sum()/ len(ppl_cleaned))*100

In [ ]:
ppl_cleaned = ppl_cleaned.dropna()

In [ ]:
ppl_cleaned.info()

### Vehicles

In [ ]:
vehicles = pd.read_csv('./data/vehicles.csv', low_memory = False)
vehicles.head()

In [ ]:
vehicles.info()

In [ ]:
vehicles.columns = vehicles.columns.str.lower()

In [ ]:
# Convert all string values in object columns to lowercase
for col in vehicles.select_dtypes(include='object').columns:
    vehicles[col] = vehicles[col].str.lower()

In [ ]:
# selecting all features with 90% or more of its values are null
high_null_features = vehicles.columns[(vehicles.isna().sum() / len(vehicles) * 100) >= 90]

In [ ]:
# creating a list of features with 90% or more null values
high_null_features_list = list(high_null_features)

In [ ]:
vehicles_cleaned = vehicles.drop(columns=high_null_features_list)

In [ ]:
vehicles_cleaned.info()

In [ ]:
vehicles_cleaned['num_passengers'].value_counts()

This is redundant information. This information does not include the driver, but this information is captured in occupant count. will drop this one. 

In [ ]:
vehicles_cleaned['unit_no'].value_counts()

This is aftermath. unhelpful. remove

In [ ]:
vehicles_cleaned['unit_type'].value_counts()

Most of the values are drivers or parked cars. this will not be useful for analysis

In [ ]:
vehicles_cleaned['make'].value_counts()

high cardinality

In [ ]:
vehicles_cleaned['model'].value_counts()

This feels like it could be helpful, but many unknowns and 'other', and very high cardinality. The important information that we'd gain from this is already included in vehicle_type. So we can drop

In [ ]:
vehicles_cleaned['vehicle_defect'].value_counts()

Most of the values are none or unknown. This will not be particularly useful for analysis. can drop

In [ ]:
vehicles_cleaned['vehicle_type'].value_counts()

In [ ]:
vehicles_cleaned['travel_direction'].value_counts()

Unhelpful for analysis. Remove

In [ ]:
vehicles_cleaned['maneuver'].value_counts()

This feature could be important as it has to do with what was happening prior to the crash.

In [ ]:
vehicles_cleaned['towed_i'].value_counts()

Aftermath; Unhelpful for analysis

In [ ]:
vehicles_cleaned['occupant_cnt'].value_counts()

In [ ]:
vehicles_cleaned['area_00_i'].value_counts()

In [ ]:
vehicles_cleaned['area_01_i'].value_counts()

It is unclear what the area_##_i features represent. They will be removed

In [ ]:
vehicles_cleaned['first_contact_point'].value_counts()

This feature could indicate

In [ ]:
vehicle_features_to_drop = ['num_passengers', 
                            'crash_unit_id',
                            'crash_date',
                            'unit_type',
                            'make', 
                            'model',
                            'vehicle_id',
                           'vehicle_defect',
                           'unit_no',
                           'lic_plate_state',
                            'vehicle_year',
                           'vehicle_use',
                           'travel_direction',
                           'towed_i',
                           'towed_by',
                           'towed_to',
                           'area_00_i',
                            'area_01_i',
                           'area_02_i', 
                           'area_03_i',
                           'area_04_i', 
                            'area_05_i',
                            'area_06_i',
                            'area_07_i',
                            'area_08_i',
                            'area_09_i',
                            'area_10_i',
                            'area_11_i',
                            'area_12_i',
                            'area_99_i', 
                           'first_contact_point']

In [ ]:
vehicles_cleaned = vehicles_cleaned.drop(columns=vehicle_features_to_drop)
vehicles_cleaned.info()